
### Twitter topic classification Hackathon
- Sentiment Classification
- Topic Modelling
  

In [1]:
## Importing all the standard libraries , 
## Pandas, numpy arrays, Regular Expression, NLTK for text processing, Sklearn for Classifiction models

import pandas as pd
import numpy as np
import sklearn
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,f1_score

### Importing the CSV file from the Local PC 
- Data set has been read in to the DataFrame variable twd(Twitter Data)
- Please paste the link of your file in the same function


In [2]:
twd=pd.read_csv(r'C:/Users/pc/Desktop/Hackathon/twcs/twcs.csv')

##### Displaying first five observations of the table. 

In [3]:
twd.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


Shape of the original Dataset

In [4]:
twd.shape

(2811774, 7)

### Here in this Data set we need the text data and either it is a good or bad statement by customer
  - ##### Hence discarding the rest of the columns

In [5]:
twd.columns

Index(['tweet_id', 'author_id', 'inbound', 'created_at', 'text',
       'response_tweet_id', 'in_response_to_tweet_id'],
      dtype='object')

##### Dropping the unecessary columns 

In [6]:
twd.drop(['tweet_id', 'author_id', 'created_at','response_tweet_id', 'in_response_to_tweet_id'], axis=1, inplace=True)

#####   Dataset after removing the columns

In [7]:
twd.head()

,inbound,text
0,False,@115712 I understand. I would like to assist y...
1,True,@sprintcare and how do you propose we do that
2,True,@sprintcare I have sent several private messag...
3,False,@115712 Please send us a Private Message so th...
4,True,@sprintcare I did.


##### Checking the total number of observations and columns present in the dataset.

In [8]:
twd.shape

(2811774, 2)

###### Only Ten thousand observations has been taken to develope the model. 
######  The New data frame variable name  is "twd_sample "(twitter data Sample) consisting 10,000 observations.
######  It restricted because of memory limitations , if it is not so, u can pass the whole data set like """ twd.iloc[:,:].copy() """


In [9]:
twd_sample=twd.iloc[2070000:2080000,:].copy()

In [10]:
twd_sample.head()

,inbound,text
2070000,True,"@idea_cares Sneha, when i call some i hear you..."
2070001,False,@336024 We apologies for the trouble. We would...
2070002,True,"@idea_cares u irritating idiots, y the fuck pr..."
2070003,False,@307570 Sorry we gave you a chance of being up...
2070004,True,@idea_cares He told me - Sir don't repeat your...


##### Checking whether our dataset consists of NaNs 

In [11]:
twd_sample.isna().sum()

inbound    0
text       0
dtype: int64

##### Counting the number of Postive and Negative comments avialable in the DataFrame  

In [12]:
twd_sample['inbound'].value_counts()

True     5428
False    4572
Name: inbound, dtype: int64

##### Writing the generalised function to remove the uncessary characters, stopwords

In [13]:
def remove_noise(input_text):
    
    lema=WordNetLemmatizer()                                ### World Lematizer 

    input_text = re.sub('<.*?>', ' ', (input_text))         ### Removing html and xml tags in text data and replacing with blanks
    input_text = str(input_text).replace("<br />", " ")     ###  Removing Line Breakage tags
    input_text = str(input_text).replace(",", "")           ### Removing punctuations and replacing with blanks
    input_text = str(input_text).replace("\'" ,"")
    input_text = str(input_text).replace("\\", "")
    input_text = str(input_text).replace("-", "")
    input_text = str(input_text).replace(".", "")
    input_text  =re.sub(r'\s+', ' ',input_text )            ### Removing multiple spaces
     
    
    input_text = re.sub("[^a-zA-Z]"," ",input_text)         ## Considering only alphabets rest is replaced by spaces
    input_text = input_text.lower()                         ## Lowering the text
    words      = input_text.split()                         ## Splitting the text
    noise_free_words =  [w for w in words if w not in set(stopwords.words('english'))] ## Removing Stop words
    lower_words=[lema.lemmatize(w) for w in noise_free_words]                          ## Applying Lematization to the words                        
    lower_words=' '.join(lower_words)
                                   
    return lower_words   

##### Splitting the Data into 70% Train, 30% Test by passing text data in to  
##### Independent variable "X" and True or False in to Targer variable "y"

In [14]:
X_train=twd_sample['text'].iloc[:7000]
y_train=twd_sample['inbound'].iloc[:7000]

X_test=twd_sample['text'].iloc[7000:10000]
y_test=twd_sample['inbound'].iloc[7000:10000]



In [15]:
print('The shape of the Training independent variable X_train',X_train.shape,'\n')
print('The shape of the Testing independent variable X_test',X_test.shape,'\n')
print('The shape of the Taining dependent variable y_train',y_train.shape,'\n')
print('The shape of the Testing dependent variable y_test',y_test.shape,'\n')

The shape of the Training independent variable X_train (7000,) 

The shape of the Testing independent variable X_test (3000,) 

The shape of the Taining dependent variable y_train (7000,) 

The shape of the Testing dependent variable y_test (3000,) 



##### Removing the noise in the Train dataset

In [16]:
X_train=X_train.apply(remove_noise)

In [17]:
X_train.head()

2070000    idea care sneha call hear irritating unlimited...
2070001    apology trouble would like inform pca reverse ...
2070002    idea care u irritating idiot fuck precall anno...
2070003    sorry gave chance upset service weve noted con...
2070004    idea care told sir dont repeat issue followed ...
Name: text, dtype: object

##### Removing the noise in the Test dataset

In [18]:
X_test=X_test.apply(remove_noise)

In [19]:
X_test.head()

2077000    im sorry inconvenience shared detail using lin...
2077001                            worst sale service amazon
2077002    certainly understand frustration id mad thank ...
2077003    amazonhelp hey ive already spoken someone cust...
2077004    im sorry filter box strange confirm order ship...
Name: text, dtype: object

##### Converting of words in the observations to matrix representation using TF-IDF Vectorizer
- Word Frequencies with TfidfVectorizer.
- This stands for “Term Frequency – Inverse Document Frequency" which are the components of the resulting -  scores assigned to each word.
- Inverse Document Frequency: This downscales words that appear a lot across documents.
- max_df,min_df, n-grams are hyper parmaters, after working on several random data, i have fixed the paramters.
- To the get best min_df,max_df, max_features, This can be done by using a Random Search CV using  Pipe Line.
- To reduce the complexity of code , i have done it seperately.

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

X_tr=X_train
X_ts=X_test
y_tr=y_train
y_ts=y_test

tf=TfidfVectorizer(max_df=0.85, min_df=3,max_features=1800,ngram_range=(1, 1), stop_words='english')

X_tr=tf.fit_transform(X_tr).toarray()

X_ts=tf.fit_transform(X_ts).toarray()

  ##### Note if X_train and X_test column "(m,n)"  "(k,n)" dimension is not matching, 
  - please change the max_features in TF-IDF Vectorizer paramter in the above.
  - Remember please give high features  to learn more from the dataset. 

In [21]:
if X_tr.shape[1]==X_ts.shape[1]:
    print('The shape of X_train is :',X_tr.shape,'\n','The shape of X_test is :',X_ts.shape)
else:
    print('Please change the max_features hyper paramter in the Tfidf vectorizer and run the code again')

The shape of X_train is : (7000, 1800) 
 The shape of X_test is : (3000, 1800)


#### NOTE:
#### X_tr, X_ts are the TF-IDF matrices trained on the repective observations

## Out of these  algorithms which ever we get  good accuracy and F1 score is choosen
- Currently my models used in this Notebook has went up to 57% accuracy on an average with different Records.

## Logistic Regression
- Machine Learning supervised algorithm to classify the positive and negative statemnts
- In this the default threshold taken as 0.5

In [22]:
from sklearn.linear_model import LogisticRegression
Logr=LogisticRegression(solver ='liblinear',class_weight='balanced')
Logr.fit(X_tr,y_tr)


LogisticRegression(class_weight='balanced', solver='liblinear')

### Test Results report  Logistic includes Accuracy, F1 Score, Confusion matrix
- Accuracy= TP+TN/(TP+TN+FP+FN)
- Precision= TP/(TP+FP)
- Recall= TP/(TP+FN)
- F1 Score= 2*(PrecisionxRecall)/(Precision+Recall)

In [23]:
y_pred=Logr.predict(X_ts)

print(classification_report(y_pred,y_ts),end=" \n")

print('Confusion Matrix :','\n',confusion_matrix(y_pred,y_ts),'\n')

print('The F1_score of the model on test _data_set:',f1_score(y_pred,y_ts),'\n')

print('The accuracy_score of the model on test _data_set:',accuracy_score(y_pred,y_ts))

              precision    recall  f1-score   support

       False       0.47      0.66      0.55       974
        True       0.80      0.64      0.71      2026

    accuracy                           0.65      3000
   macro avg       0.63      0.65      0.63      3000
weighted avg       0.69      0.65      0.66      3000
 
Confusion Matrix : 
 [[ 645  329]
 [ 733 1293]] 

The F1_score of the model on test _data_set: 0.7088815789473685 

The accuracy_score of the model on test _data_set: 0.646


## Decision Tree classifier
- _Intially performing Randomized Search on the pramaters available in the DecisionTree Model_.
- _Also in Randomzed Search N-iterations has been taken only 5 due to memory constrains_.
-  _u can good to go with higher iterations_.
- _After getting best paramters those have to be substituted in to the algorithm_.

In [24]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4, 5],'max_depth':list(range(8, 100,2))}
dt = DecisionTreeClassifier()
clf=RandomizedSearchCV(estimator = dt, param_distributions = params, n_iter = 8, cv = 5,\
                               verbose=3, random_state=42, n_jobs = -1)

clf.fit(X_tr, y_tr)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   59.6s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  2.0min finished


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=8,
                   n_jobs=-1,
                   param_distributions={'max_depth': [8, 10, 12, 14, 16, 18, 20,
                                                      22, 24, 26, 28, 30, 32,
                                                      34, 36, 38, 40, 42, 44,
                                                      46, 48, 50, 52, 54, 56,
                                                      58, 60, 62, 64, 66, ...],
                                        'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8,
                                                           9, 10, 11, 12, 13,
                                                           14, 15, 16, 17, 18,
                                                           19, 20, 21, 22, 23,
                                                           24, 25, 26, 27, 28,
                                                           29, 30, 31, ...],
                                

#### Use the below  best classifer fit for the Decision tree

In [25]:
clf.best_estimator_


DecisionTreeClassifier(max_depth=64, max_leaf_nodes=79)

In [26]:
from sklearn.tree import DecisionTreeClassifier

Dt=DecisionTreeClassifier(max_depth=64, max_leaf_nodes=79)

Dt.fit(X_tr,y_tr)


DecisionTreeClassifier(max_depth=64, max_leaf_nodes=79)

### Test Results report  Decision Tree includes Accuracy, F1 Score, Confusion matrix

In [27]:
y_pred=Dt.predict(X_ts)

print(classification_report(y_pred,y_ts),end=" \n")

print('Confusion Matrix :','\n',confusion_matrix(y_pred,y_ts),'\n')

print('The F1_score of the model on test _data_set:',f1_score(y_pred,y_ts),'\n')

print('The accuracy_score of the model on test _data_set:',accuracy_score(y_pred,y_ts))

              precision    recall  f1-score   support

       False       0.42      0.65      0.51       893
        True       0.81      0.62      0.70      2107

    accuracy                           0.63      3000
   macro avg       0.62      0.64      0.61      3000
weighted avg       0.69      0.63      0.65      3000
 
Confusion Matrix : 
 [[ 584  309]
 [ 794 1313]] 

The F1_score of the model on test _data_set: 0.7042102440332528 

The accuracy_score of the model on test _data_set: 0.6323333333333333


##### Cross validation on Train Dataset
- _Checking the cross validation on the Train dataset_.
- _The model is performing well on Train data set because it has 70% of observations_.

In [60]:
from sklearn.model_selection import cross_val_score 
Dt=DecisionTreeClassifier()
accuracies = cross_val_score(estimator = Dt, X = X_tr,\
     y = y_tr, cv = 5,verbose=2)
print("Accuracy Mean {} Accuracy Variance \
     {}".format(accuracies.mean(),accuracies.std()))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total=  16.4s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.3s remaining:    0.0s


[CV] ................................................. , total=  19.6s
[CV]  ................................................................
[CV] ................................................. , total=  14.2s
[CV]  ................................................................
[CV] ................................................. , total=  15.6s
[CV]  ................................................................
[CV] ................................................. , total=  16.2s
Accuracy Mean 0.8344285714285714 Accuracy Variance      0.0024074713637646616


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.4min finished


## Random Forest classifier
- _Intially performing Randomized Search on the pramaters available in the Random forest Model_.
- _Also in Randomzed Search N-iterations has been taken only 5 due to memory constrains_. 
-  _You can good to go with higher iterations_.
- _After getting best paramters those have to be substituted in to the algorithm_.

In [29]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 100)]
# Number of features to consider at every split
max_features = [int(x) for x in np.linspace(start = 20, stop = 100, num = 10)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(2, 100, num = 1)]

# Minimum number of samples required to split a node
min_samples_split = [2, 3,4,5,6, 7,8,9,10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [ 2, 3,4,5,6,7,8,9]
# Method of selecting samples for training each tree
bootstrap = [True, False]


# Create the random grid
params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = params, n_iter = 5, cv = 5,\
                               verbose=3, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_tr, y_tr)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  2.0min remaining:   38.4s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  2.2min finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=5,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [2],
                                        'max_features': [20, 28, 37, 46, 55, 64,
                                                         73, 82, 91, 100],
                                        'min_samples_leaf': [2, 3, 4, 5, 6, 7,
                                                             8, 9],
                                        'min_samples_split': [2, 3, 4, 5, 6, 7,
                                                              8, 9, 10],
                                        'n_estimators': [100, 109, 118, 127,
                                                         136, 145, 154, 163,
                                                         172, 181, 190, 200,
                                                         209, 218, 227, 236,
         

In [38]:
rf_random.best_estimator_

RandomForestClassifier(bootstrap=False, max_depth=2, max_features=91,
                       min_samples_leaf=4, min_samples_split=7,
                       n_estimators=390)

In [31]:
from sklearn.ensemble import RandomForestClassifier

RF=RandomForestClassifier(bootstrap=False, max_depth=2, max_features=91,
                       min_samples_leaf=4, min_samples_split=7,
                       n_estimators=390)
RF.fit(X_tr,y_tr)


RandomForestClassifier(bootstrap=False, max_depth=2, max_features=91,
                       min_samples_leaf=4, min_samples_split=7,
                       n_estimators=390)

### Test Results report  Random Forest includes Accuracy, F1 Score, Confusion matrix

In [32]:
y_pred=RF.predict(X_ts)

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

print(classification_report(y_pred,y_ts),end=" \n")

print('Confusion Matrix :','\n',confusion_matrix(y_pred,y_ts),'\n')

print('The F1_score of the model on test _data_set:',f1_score(y_pred,y_ts),'\n')

print('The accuracy_score of the model on test _data_set:',accuracy_score(y_pred,y_ts))

              precision    recall  f1-score   support

       False       0.01      0.86      0.03        22
        True       1.00      0.54      0.70      2978

    accuracy                           0.55      3000
   macro avg       0.51      0.70      0.37      3000
weighted avg       0.99      0.55      0.70      3000
 
Confusion Matrix : 
 [[  19    3]
 [1359 1619]] 

The F1_score of the model on test _data_set: 0.7039130434782609 

The accuracy_score of the model on test _data_set: 0.546


In [117]:
from sklearn.model_selection import cross_val_score 
accuracies = cross_val_score(estimator = RF, X = X_tr,\
     y = y_tr, cv = 5)
print("Accuracy Mean {} Accuracy Variance \
     {}".format(accuracies.mean(),accuracies.std()))


Accuracy Mean 0.91425 Accuracy Variance      0.007988272654335235


## Support Vector Machines

In [33]:
from sklearn.svm import SVC

svc = SVC(kernel='rbf',C=1)
svc.fit(X_tr,y_tr)


SVC(C=1)

## Test Results Support Vector Machines Accuracy,F1_Score Confusion Matrix

In [34]:
y_pred=svc.predict(X_ts)


print(classification_report(y_pred,y_ts),end=" \n")

print('Confusion Matrix :','\n',confusion_matrix(y_pred,y_ts),'\n')

print('The F1_score of the model on test _data_set:',f1_score(y_pred,y_ts),'\n')

print('The accuracy_score of the model on test _data_set:',accuracy_score(y_pred,y_ts))

              precision    recall  f1-score   support

       False       0.43      0.66      0.52       891
        True       0.81      0.62      0.71      2109

    accuracy                           0.64      3000
   macro avg       0.62      0.64      0.61      3000
weighted avg       0.70      0.64      0.65      3000
 
Confusion Matrix : 
 [[ 587  304]
 [ 791 1318]] 

The F1_score of the model on test _data_set: 0.706512999195926 

The accuracy_score of the model on test _data_set: 0.635


## Naive bayes MultinomialNB

In [35]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB

navby=MultinomialNB()
navby.fit(X_tr,y_tr)


MultinomialNB()

## Test Results MultiNomialNB Accuracy,F1_Score< Confusion Matrix

In [36]:
y_pred=navby.predict(X_ts)

print(classification_report(y_pred,y_ts),end=" \n")

print('Confusion Matrix :','\n',confusion_matrix(y_pred,y_ts),'\n')

print('The F1_score of the model on test _data_set:',f1_score(y_pred,y_ts),'\n')

print('The accuracy_score of the model on test _data_set:',accuracy_score(y_pred,y_ts))

              precision    recall  f1-score   support

       False       0.39      0.62      0.48       871
        True       0.80      0.61      0.69      2129

    accuracy                           0.61      3000
   macro avg       0.59      0.61      0.58      3000
weighted avg       0.68      0.61      0.63      3000
 
Confusion Matrix : 
 [[ 539  332]
 [ 839 1290]] 

The F1_score of the model on test _data_set: 0.6878165822447347 

The accuracy_score of the model on test _data_set: 0.6096666666666667


## Cross validation for Naive bayes

In [63]:
from sklearn.model_selection import cross_val_score 
accuracies = cross_val_score(estimator = navby, X = X_tr,\
     y = y_tr, cv = 7,verbose=2)
print("Accuracy Mean {} Accuracy Variance \
     {}".format(accuracies.mean(),accuracies.std()))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
Accura

[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.8s finished


## XG Boost classifier
- _Intially performing Randomized Search on the pramaters available in the XG Boost Model.
- _Also in Randomzed Search N_iterations has been taken only 5 due to memory constrains_. 
-  _You can good to go with higher iterations_.
- _After getting best paramters those have to be substituted in to the algorithm_.

In [77]:
## Hyper Parameter Optimization
import xgboost as xgb
params={
 "learning_rate"    : [0.01,0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ],
    "reg_alpha"     : [1,3,5,7,10],
     "reg_lambda"   : [1,3,5,7,10]
}

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
classifier=xgb.XGBClassifier()

random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)
random_search.fit(X_tr,y_tr)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  3.8min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  4.4min finished


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           subsample=None, tree_method=None,
                                           validate_parameters=None,
                                   

- Best Estimator after hyper paramter tuning
- In this estimator remove "missing=nan" paramter

In [78]:
random_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.25, max_delta_step=0, max_depth=10,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=1, reg_lambda=10, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [80]:
import xgboost as xgb

clf=xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.25, max_delta_step=0, max_depth=10,
              min_child_weight=3,  monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=1, reg_lambda=10, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)



clf.fit(X_tr,y_tr)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.25, max_delta_step=0, max_depth=10,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=1, reg_lambda=10, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

## Test Results XG Boost Accuracy,F1_Score Confusion Matrix

In [81]:
y_pred=clf.predict(X_ts)


print(classification_report(y_pred,y_ts),end=" \n")

print('Confusion Matrix :','\n',confusion_matrix(y_pred,y_ts),'\n')


print('The f1_score of the model:',f1_score(y_pred,y_ts))


print('The accuracy_score of the model:',accuracy_score(y_pred,y_ts))


              precision    recall  f1-score   support

       False       0.35      0.68      0.46       709
        True       0.86      0.61      0.71      2291

    accuracy                           0.62      3000
   macro avg       0.60      0.64      0.59      3000
weighted avg       0.74      0.62      0.65      3000
 
Confusion Matrix : 
 [[ 481  228]
 [ 897 1394]] 

The f1_score of the model: 0.7124968055200613
The accuracy_score of the model: 0.625


##### Cross validarion on Train dataset XG Boost

In [63]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(random_search, X_tr, y_tr, cv=5, scoring='f1_macro')
scores.mean()

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  2.6min remaining:   49.5s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  3.0min finished


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  2.4min remaining:   45.3s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  3.3min finished


Fitting 5 folds for each of 5 candidates, totalling 25 fits


...

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  3.2min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  3.6min finished


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  4.7min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  5.2min finished


0.8812935332752637

### Topic Modelling

- All base libraries are imported

In [56]:
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [57]:
twd_sample.head()

,inbound,text
2070000,True,"@idea_cares Sneha, when i call some i hear you..."
2070001,False,@336024 We apologies for the trouble. We would...
2070002,True,"@idea_cares u irritating idiots, y the fuck pr..."
2070003,False,@307570 Sorry we gave you a chance of being up...
2070004,True,@idea_cares He told me - Sir don't repeat your...


- Checking the total number of observations and features

In [58]:
twd_sample.shape

(10000, 2)

In [59]:
twd_sample['text']=twd_sample['text'].apply(remove_noise)

- Text data after cleaning

In [60]:
twd_sample.head()

,inbound,text
2070000,True,idea care sneha call hear irritating unlimited...
2070001,False,apology trouble would like inform pca reverse ...
2070002,True,idea care u irritating idiot fuck precall anno...
2070003,False,sorry gave chance upset service weve noted con...
2070004,True,idea care told sir dont repeat issue followed ...


###### Considering only the Negative topics in the data set

In [61]:
twd_sample_negative=twd_sample[twd_sample['inbound']==False]
twd_sample_negative.head()

,inbound,text
2070001,False,apology trouble would like inform pca reverse ...
2070003,False,sorry gave chance upset service weve noted con...
2070008,False,troy sorry disappointing experience hope give ...
2070010,False,time feed need molly deliciousness awaits
2070012,False,sorry inconvenience checked offer applicable h...


###### Considering only the Positive topics in the data set

In [62]:
twd_sample_positive=twd_sample[twd_sample['inbound']==True]
twd_sample_positive.head()

,inbound,text
2070000,True,idea care sneha call hear irritating unlimited...
2070002,True,idea care u irritating idiot fuck precall anno...
2070004,True,idea care told sir dont repeat issue followed ...
2070005,True,idea care darshan tried seek help one supervis...
2070006,True,idea care darshan extremely serious issue firs...


In [63]:
def tokenize(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    
    filtered_tokens = []
    
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    
    return filtered_tokens

- Intial check of the function

In [64]:
tokenize(twd_sample_negative.iloc[0,1])

['apology',
 'trouble',
 'would',
 'like',
 'inform',
 'pca',
 'reverse',
 'pca',
 'playing',
 'someone',
 'try',
 'call',
 'regardssneha']

#### Tfidf Vectorizer with unigrams


In [68]:
term_idf_vectorizer      =     TfidfVectorizer(stop_words = 'english',max_features=2000, min_df=0.0005,\
                                      max_df=0.99,tokenizer = tokenize)
term_idf_matrix          =     term_idf_vectorizer.fit_transform(twd_sample_negative.text) 
term_idf_feature_names   =     term_idf_vectorizer.get_feature_names()


## Non Negative Matrix Factorization
- NNMF is an unsuperviser algorithm that performs dimensionality reduction and clustering
- We use this in conjuction with TF-IDF to model topics across documents


In [69]:
nmf_model=NMF(n_components=15,init='nndsvd',random_state=22)
nmf_model.fit(term_idf_matrix)

NMF(init='nndsvd', n_components=15, random_state=22)

#### Printing out the Top 15 Topics involved in the entire documents

In [70]:
for index, topic in enumerate(nmf_model.components_):
    print(f"The top 15 words for topic # {index}")
    print([term_idf_feature_names[i] for i in topic.argsort()[-15:]])
    print('\n')

The top 15 words for topic # 0
['r', 'info', 'use', 'f', 'information', 'woafwenp', 'check', 'j', 'h', 'wkjhdxwgrq', 'c', 'u', 'link', 'tco', 'http']


The top 15 words for topic # 1
['linked', 'associated', 'apologize', 'right', 'look', 'trouble', 'amp', 'connect', 'send', 'account', 'u', 'follow', 'dm', 'address', 'email']


The top 15 words for topic # 2
['device', 'happened', 'aw', 'try', 'sure', 'follow', 'resolved', 'additional', 'twitter', 'going', 'question', 'letting', 'u', 'let', 'know']


The top 15 words for topic # 3
['frustration', 'oh', 'way', 'really', 'order', 'tell', 'caused', 'trouble', 'inconvenience', 'delay', 'experience', 'store', 'im', 'hear', 'sorry']


The top 15 words for topic # 4
['shortly', 'right', 'contact', 'member', 'possible', 'support', 'patience', 'share', 'soon', 'appreciate', 'feedback', 'connect', 'touch', 'note', 'team']


The top 15 words for topic # 5
['ah', 'fine', 'ticket', 'speak', 'wish', 'andy', 'nice', 'colleague', 'start', 'longer', 'ha

#### TF-IDF Vectorizer with bigrams and tri grams

In [71]:
topic_results=nmf_model.transform(term_idf_matrix)

In [72]:
term_idf_vectorizer     = TfidfVectorizer(max_df=0.99, max_features=2000,min_df=0.0005,\
                                          stop_words='english', use_idf=True, tokenizer=tokenize, ngram_range=(2,3))
term_idf_matrix         = term_idf_vectorizer.fit_transform(twd_sample_negative.text) 
term_idf_feature_names  = term_idf_vectorizer.get_feature_names()
term_idf_matrix.shape

nmf_model=NMF(n_components=15,init='nndsvd',random_state=22)
nmf_model.fit(term_idf_matrix)

NMF(init='nndsvd', n_components=15, random_state=22)

In [73]:
for index, topic in enumerate(nmf_model.components_):
    print(f"The top 15 words for topic # {index}")
    print([term_idf_feature_names[i] for i in topic.argsort()[-15:]])
    print('\n')

The top 15 words for topic # 0
['help http', 'help http tco', 'qui http tco', 'information http', 'information http tco', 'check http', 'check http tco', 'dm http tco', 'dm http', 'hope help', 'http tco c', 'tco c', 'link http', 'link http tco', 'http tco']


The top 15 words for topic # 1
['whats going', 'need assistance', 'help let', 'need help', 'help let u', 'know assistance', 'u know question', 'know question', 'know help', 'u know help', 'know need', 'u know need', 'u know', 'let u know', 'let u']


The top 15 words for topic # 2
['dm assist', 'u dm assist', 'email address follow', 'address follow', 'address connect', 'email address connect', 'help send', 'help send u', 'u dm email', 'dm email', 'dm email address', 'email address', 'send u', 'send u dm', 'u dm']


The top 15 words for topic # 3
['let know', 'im sorry inconvenience', 'im sorry delay', 'sorry inconvenience caused', 'tell u', 'hi im', 'hi im sorry', 'oh im', 'oh im sorry', 'sorry frustration', 'im sorry frustration'

In [74]:
twd_sample_negative['Topic']=topic_results.argmax(axis=1)

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [75]:
twd_sample_negative.head()

,inbound,text,Topic
2070001,False,apology trouble would like inform pca reverse ...,10
2070003,False,sorry gave chance upset service weve noted con...,3
2070008,False,troy sorry disappointing experience hope give ...,3
2070010,False,time feed need molly deliciousness awaits,14
2070012,False,sorry inconvenience checked offer applicable h...,3


## People related to topic 4 are segregated 

In [76]:
print(twd_sample_negative[twd_sample_negative['Topic']==4].head(),'\n')

print('Total Number of people Related to topic "4" is',twd_sample_negative[twd_sample_negative['Topic']==4].shape[0])

         inbound                                               text  Topic
2070045    False  apology trouble request share complete address...      4
2070084    False  sorry problem service understand frustrating f...      4
2070122    False  oh dear sorry chris feedback logged passed mar...      4
2070150    False  hi richard escalated store maintenance team sm...      4
2070160    False                   get food online team look smaira      4 

Total Number of people Related to topic "4" is 338
